<a href="https://colab.research.google.com/github/aminghafari1/lcaging-fmri/blob/main/outlier_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## For now, the idea is to make two 3d matrices, one for auditory and one for visual.
## In each layer of the 3d mat we will put one subject with its runs.
## This is to know their place later that they are detected as outliers, so we know where they come from

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import glob
import os
def substring_after(s,delim):
  return s.partition(delim)[2]
def substring_before(s,delim):
  return s.partition(delim)[0]
import scipy.io as sio
import os
from scipy.stats import zscore
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import csv

Mounted at /content/gdrive


In [ ]:

## read the csv files for the auditory cortex
base_dir=glob.glob('/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/')
base_dir=base_dir[0]
prefix = 'BAP'
sub_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f)) and f.startswith(prefix)]
sub_folders = [base_dir + i for i in sub_folders]
auditory_files=[]
visual_files=[]
accuracy_results="updated_auditory_accuracy.csv"
for dir in sub_folders:
    aud_dir = os.path.join(dir, 'auditory')
    vis_dir = os.path.join(dir, 'visual')
    if os.path.exists(aud_dir):
       accuracy_csv=os.path.join(aud_dir, accuracy_results) 
       if os.path.exists(accuracy_csv) == True:
         auditory_files.append(accuracy_csv)
number_of_run=5 ##each subject has 5 runs
number_of_stimulus_levels=5 ##each run has 5 different stimulus levels
accuracy_matrix = np.zeros((len(auditory_files), number_of_run, number_of_stimulus_levels))
layer_subjects={}
for i in range(len(auditory_files)):
  temp=substring_before(auditory_files[i],"/audit")
  subject_index=substring_after(temp,"MRI/")
  layer_subjects[i]=subject_index
  accuracy_csv=auditory_files[i]
  with open(accuracy_csv, 'r') as file:
   reader=csv.reader(file)
   matrix = np.array(list(reader))
   matrix[matrix == ''] = '0'
   matrix_float = matrix.astype('float')
   matrix_float = matrix_float[1:, 1:]
   accuracy_matrix[i,:,:]=matrix_float




In [ ]:
## plotting the results
x=[0,4,8,16,32]
import time
from IPython.display import clear_output
for i in range(accuracy_matrix.shape[0]):
    # Loop through each row of the layer
    for j in range(accuracy_matrix.shape[1]):
        # Extract the y values for the current row
        figure, ax = plt.subplots(figsize=(10, 8))
        y = accuracy_matrix[i, j, :]
        line1, = ax.plot(x, y)
        # Plot x by y
        plt.close()
        plt.plot(x, y)
        plt.grid(True)
        plt.title(f" {layer_subjects[i]}, Run {j}")
        plt.show()
        figure.canvas.draw()
        figure.canvas.flush_events()
        time.sleep(1.5)
        clear_output(wait=True)

        

In [ ]:
orig_shape=accuracy_matrix.shape
data=np.reshape(accuracy_matrix,(orig_shape[0]*orig_shape[1],orig_shape[2]))


In [ ]:
## Method 1: Mahalanobis Distance
## calculate the mean of the data
import time
from IPython.display import clear_output
contrast=[0,4,8,16,32]
mean=np.mean(data,axis=0)
## calculate the malahobis distance between each row and the mean and save it in a matrix named mahal
mahal=np.zeros((len(data),1))
for i in range(len(data)):
    mahal[i]=np.sqrt((data[i]-mean).T.dot(np.linalg.inv(np.cov(data.T))).dot((data[i]-mean)))
## plot the mahalobis distance
plt.plot(mahal)
plt.grid(True)
plt.title(f" mahalobis distance")
plt.show()
time.sleep(2)
clear_output(wait=True)
## find the outliers based on the mahalobis distance
mahal_outliers=[]
for i in range(len(mahal)):
    if mahal[i]>np.mean(mahal)+2*np.std(mahal):
        mahal_outliers.append(i)
## plot the outliers
for i in mahal_outliers:
    print("i is %d" %(i))
    figure, ax = plt.subplots(figsize=(10, 8))
    y = data[i]
    x=contrast
    line1, = ax.plot(x, y)
        # Plot x by y
    plt.close()
    plt.plot(x, y)
    plt.grid(True)
    plt.title(f" run number {[i]}")
    plt.show()
    figure.canvas.draw()
    figure.canvas.flush_events()
    time.sleep(2)
    clear_output(wait=True)

In [ ]:
## Method 2: Frechet Distance
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import frechet_distance
mean=np.mean(data,axis=0)

frechet=np.zeros(len(data))


In [ ]:
## Method 2: 1-D PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(data)
data_pca=pca.transform(data)
## plot the pca
plt.plot(data_pca)
plt.grid(True)
plt.title(f" pca")
plt.show()
time.sleep(2)
clear_output(wait=True)
## find the outliers based on the pca
pca_outliers=[] 
for i in range(len(data_pca)):
    if data_pca[i]>np.mean(data_pca)+2*np.std(data_pca):
        pca_outliers.append(i)
## plot the outliers
for i in pca_outliers:
    print("i is %d" %(i))
    figure, ax = plt.subplots(figsize=(10, 8))
    y = data[i]
    x=contrast
    line1, = ax.plot(x, y)
        # Plot x by y
    plt.close()
    plt.plot(x, y)
    plt.grid(True)
    plt.title(f" run number {[i]}")
    plt.show()
    figure.canvas.draw()
    figure.canvas.flush_events()
    time.sleep(2)
    clear_output(wait=True)

In [ ]:
## Give me 2-d PCA of the data
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(data)
data_pca=pca.transform(data)
## plot the pca
plt.scatter(data_pca[:,0],data_pca[:,1])
plt.grid(True)
plt.title(f" pca")
plt.show()
time.sleep(2)
clear_output(wait=True)
## find the outliers based on the pca
pca_outliers=[]
for i in range(len(data_pca)):
    if data_pca[i,0]>np.mean(data_pca[:,0])+2*np.std(data_pca[:,0]):
        pca_outliers.append(i)
## plot the outliers
for i in pca_outliers:
    print("i is %d" %(i))
    figure, ax = plt.subplots(figsize=(10, 8))
    y = data[i]
    x=contrast
    line1, = ax.plot(x, y)
        # Plot x by y
    plt.close()
    plt.plot(x, y)
    plt.grid(True)
    plt.title(f" run number {[i]}")
    plt.show()
    figure.canvas.draw()
    figure.canvas.flush_events()
    time.sleep(2)
    clear_output(wait=True)